# Simple Fisher Matrix Computation

This tutorial demonstrates how to compute a basic Fisher information matrix using StableEMRIFisher with FastKerrEccentricEquatorialFlux waveforms.

We'll show how to:
- Set up waveform parameters
- Configure the StableEMRIFisher class
- Compute a Fisher matrix for a subset of parameters
- Extract parameter uncertainties

## Import Required Packages

First, let's import the necessary packages for EMRI waveform generation and Fisher matrix computation.

In [ ]:
# Import relevant EMRI packages
from few.waveform import (
    GenerateEMRIWaveform,
    FastKerrEccentricEquatorialFlux,
)
from stableemrifisher.fisher import StableEMRIFisher

import numpy as np

## Define Waveform Parameters

Set up the physical parameters for our EMRI system and waveform generation settings.

In [ ]:
# Waveform generation parameters
dt = 5.0   # Time step (seconds)
T = 0.01   # Total observation time (years)

# Physical parameters for the EMRI system
wave_params = {
    "m1": 1e6,        # Primary mass (solar masses)
    "m2": 1e1,        # Secondary mass (solar masses) 
    "a": 0.9,         # Dimensionless spin parameter
    "p0": 10,         # Initial semi-latus rectum
    "e0": 0.4,        # Initial eccentricity
    "xI0": 1.0,       # Initial inclination (cos(inclination))
    "dist": 1.0,      # Distance to source (Gpc)
    "qS": 0.2,        # Sky location polar angle
    "phiS": 0.8,      # Sky location azimuthal angle
    "qK": 1.6,        # Kerr spin polar angle
    "phiK": 1.5,      # Kerr spin azimuthal angle
    "Phi_phi0": 2.0,  # Initial phase in phi
    "Phi_theta0": 0.0, # Initial phase in theta
    "Phi_r0": 3.0,    # Initial phase in r
}

print(f"EMRI system parameters:")
for key, value in wave_params.items():
    print(f"  {key}: {value}")

## Configure Waveform Generation

Set up the waveform class and generator with appropriate accuracy settings.

In [ ]:
# Waveform class configuration
waveform_class_kwargs = {
    "inspiral_kwargs": {
        "err": 1e-11,  # Integration error tolerance
    },
    "mode_selector_kwargs": {
        "mode_selection_threshold": 1e-5  # Threshold for mode selection
    },
}

# Waveform generator configuration
waveform_generator = GenerateEMRIWaveform
waveform_generator_kwargs = {
    "return_list": False, 
    "frame": "detector"  # Generate detector frame waveforms
}

print("Waveform generation configured for detector frame with high accuracy.")

## Initialize StableEMRIFisher

Create the Fisher matrix computation object with derivative settings.

In [ ]:
# Fisher matrix computation settings
der_order = 4      # Order of finite difference derivatives
Ndelta = 8         # Number of step sizes to test for stability
stability_plot = False  # Don't show stability plots for this demo

# Initialize the StableEMRIFisher object
sef = StableEMRIFisher(
    waveform_class=FastKerrEccentricEquatorialFlux,
    waveform_class_kwargs=waveform_class_kwargs,
    waveform_generator=GenerateEMRIWaveform,
    waveform_generator_kwargs=waveform_generator_kwargs,
    dt=dt,
    T=T,
    der_order=der_order,
    Ndelta=Ndelta,
    deriv_type="stable",  # Use stable derivative computation
)

print(f"StableEMRIFisher initialized with {der_order}th order derivatives")
print(f"Using stable derivative method with {Ndelta} step sizes for optimization")

## Define Parameters for Fisher Matrix

Select which parameters to include in the Fisher matrix and specify step size ranges for derivative computation.

In [ ]:
# Parameters to include in Fisher matrix
param_names = [
    "m1",  # Primary mass
    "m2",  # Secondary mass
    "a",   # Spin parameter
]

# Define step size ranges for each parameter (for derivative stability)
delta_range = dict(
    m1=np.geomspace(1e2, 1e-3, Ndelta),    # Mass parameters
    m2=np.geomspace(1e-3, 1e-8, Ndelta),   # Secondary mass (smaller)
    a=np.geomspace(1e-4, 1e-9, Ndelta),    # Spin parameter
)

print(f"Computing Fisher matrix for parameters: {param_names}")
print(f"Step size ranges defined for derivative optimization")

## Compute Fisher Matrix

Now we compute the Fisher information matrix. This will automatically:
1. Find optimal step sizes for numerical derivatives
2. Compute derivatives for each parameter
3. Calculate the Fisher matrix elements

In [ ]:
# Compute the Fisher matrix
print("Computing Fisher matrix...")
print("This may take a moment as we optimize derivative step sizes...")

fisher_matrix = sef(
    wave_params, 
    param_names=param_names, 
    delta_range=delta_range
)

print(f"\nFisher matrix computed successfully!")
print(f"Shape: {fisher_matrix.shape}")
print(f"\nFisher Matrix:")
print(fisher_matrix)

## Extract Parameter Uncertainties

The parameter covariance matrix is the inverse of the Fisher matrix. The diagonal elements give us the parameter variances (uncertainties squared).

In [ ]:
# Compute parameter covariance matrix
param_cov = np.linalg.inv(fisher_matrix)

print("Parameter Measurement Uncertainties (1-sigma):")
print("=" * 45)

for k, param in enumerate(param_names):
    uncertainty = param_cov[k, k] ** 0.5  # Standard deviation
    relative_error = uncertainty / wave_params[param] * 100
    
    print(f"{param:>4}: {uncertainty:.2e} ({relative_error:.2f}% relative error)")

print("\nCovariance Matrix:")
print(param_cov)

## Summary

This tutorial demonstrated the basic workflow for computing Fisher information matrices with StableEMRIFisher:

1. **Parameter Setup**: Define physical parameters for the EMRI system
2. **Waveform Configuration**: Set up waveform generation with appropriate accuracy
3. **Fisher Object**: Initialize StableEMRIFisher with derivative settings
4. **Computation**: Compute the Fisher matrix with automatic step size optimization
5. **Results**: Extract parameter uncertainties from the covariance matrix

The stable derivative method automatically finds optimal finite difference step sizes, ensuring robust and accurate Fisher matrix computation for parameter estimation studies.